In [1]:
include("../LiPoSID.jl")
using QuantumOptics
basis = NLevelBasis(2)
using LinearAlgebra
using HDF5
using DynamicPolynomials
using Plots
using Dates

using Optim
using Random

using Statistics

Consider simple Lindblad master equation with two dissipators:

$
    \frac{d\rho}{dt} = -\frac{i}{\hbar} [H,\rho(t)] + \mathcal{D}[\rho(t)] = - \frac{i}{\hbar}[H, \rho]+\sum_{\ell=1}^{s-1}\left[J_\ell \rho J_\ell^\dagger - \frac{1}{2}\left\{ J_\ell^\dagger J_\ell, \rho \right\} \right],
$

where Hamiltonian is hermitian with one of the diagonal elemnets set to zero


$
    H = \begin{pmatrix} w & 0 \\ 0 & 0
   \end{pmatrix}
$

 withot loss of generality we can possibly look for jump operator of the form:

$
J_1 = \begin{pmatrix} 0 & a \\ 0 & 0 
   \end{pmatrix} = 
   \begin{pmatrix} 0 & \sqrt{\gamma_1} \\ 0 & 0 
   \end{pmatrix}
$



In [2]:
# Define polynomial variables
@polyvar w α β r 

Hˢʸᵐᵇ = [ w   0
          0   0. ]

#Hˢʸᵐᵇ = [ w           α + im* β
#          α + im* β   0.        ]

J₁ˢʸᵐᵇ =  [ 0   r
            0   0. ]  # relaxation

Jˢʸᵐᵇ = [J₁ˢʸᵐᵇ ]


function read_timeevolution(file_name, state, γ)
    h5open(file_name, "r") do file
        ρᵧ = read(file[state][string(γ)])
        t = ρᵧ["t"]
        ρ₀₀ = ρᵧ["p0"]; Re_ρ₀₁ = ρᵧ["s_re"];  Im_ρ₀₁ = ρᵧ["s_im"]
        ρ_series = []
        t_series = []

        for i in 1:length(t)
            ρᵢ= [ ρ₀₀[i]                      Re_ρ₀₁[i] + im * Im_ρ₀₁[i]
                  Re_ρ₀₁[i] - im * Im_ρ₀₁[i]  1 - ρ₀₀[i]                 ]
            push!(ρ_series, convert(Matrix{ComplexF64}, ρᵢ))
            push!(t_series, convert(Float64, t[i]))
        end
        return(t_series, ρ_series)
    end
end


function g_objective(γᵢ, Hˢʸᵐᵇ, Jˢʸᵐᵇ, file_name, states)

    objₑₓ = 0

    for state in states # loop over initial states

        tᵗʳⁿ, ρᵗʳⁿ = read_timeevolution(file_name, state, γᵢ)
        
        #if length(tᵗʳⁿ) > 1200 end_train = 1200 else end_train = length(tᵗʳⁿ) end
            
        ρᵗʳⁿ = convert(Vector{Matrix{ComplexF64}}, ρᵗʳⁿ) #[1:end_train])
        
        tᵗʳⁿ = convert(Vector{Float64}, tᵗʳⁿ)#[1:end_train])

        objₑₓ += LiPoSID.simpson_obj(ρᵗʳⁿ, tᵗʳⁿ,  Hˢʸᵐᵇ, Jˢʸᵐᵇ )
        
    end # of files (initial states) loop

    return(objₑₓ)
end

g_objective (generic function with 1 method)

In [3]:

function scaling_poly(p::Polynomial)

    X = transpose(hcat([exponents(t) for t in terms(p)]...))

    # Get the scaling via linear regression
    scaling = X \ log.(abs.(coefficients(p)))

    exp.(abs.(scaling)) # 
end

function local_rand_min(p)

    pd = p / maximum(abs.(coefficients(p)))

    # find variable scaling
    scale = scaling_poly(pd)

    # scale the polynomial
    p_scaled = subs(pd, variables(pd) => scale .* variables(pd))

    num_iterations = 100

    # Initialize the best minimizer and the minimum value
    best_minimizer = nothing
    best_min_value = Inf

    num_of_variables = length(variables(pd))

    for _ in 1:num_iterations
        # Generate a random initial condition
        initial_point = rand(num_of_variables).*250

        # Run local optimization
        result = Optim.optimize(p_scaled, initial_point, BFGS())
        #println(Optim.minimum(result))

        # Update the best minimizer if a better one is found
        if Optim.minimum(result) < best_min_value
            
            best_minimizer = Optim.minimizer(result)
            best_min_value = Optim.minimum(result)
            
        end

    end

    best_minimizer = abs.(best_minimizer) # to make gamma positive

    minimizer_scaled = scale .* best_minimizer

    solution = variables(p_scaled) => minimizer_scaled

end

function TrDist(ρ₁, ρ₂)
    A = ρ₁-ρ₂
    D = tr(sqrt(A'*A))/2
    if abs(imag(D))>1e-6
        throw(DomainError(D, "Trace distance is complex number"))
    else
        return(real(D))
    end
end 

TrDist (generic function with 1 method)

In [4]:
#tests_dir = "../TESTS/"
tests_dir = ""

ρᵍ₀ = [ 1 0
        0 0 ]    # state to measure initial distance from

dodeca_10_states = ["D"*string(n) for n=1:10];

basis_states = ["B"*string(n) for n=1:4];

train_states = basis_states 
test_states = dodeca_10_states

all_files = vcat(train_states, test_states);

In [5]:
γ = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133", "79.477", "251.33"]

date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

evol_data_file_name = "../DATA/ALL_GAMMAS_B4_D10.h5"

op_num = 1

tests_data_file_name = "POP_NEW_LME_AMZ1_op"*string(op_num)*"_random_trn4_tst10_"*date_and_time_string * ".h5"

FminGammas = []
FmedianGammas = []
FmeanGammas = []
Omegas = []


for γᵢ in γ

    F = []

    println("γ =  "*γᵢ)

    poly = g_objective(γᵢ, Hˢʸᵐᵇ, Jˢʸᵐᵇ[1:op_num], evol_data_file_name, train_states)
   
    sol = local_rand_min(poly)

    Hˢⁱᵈ = subs(Hˢʸᵐᵇ, sol)

    omega = subs(w, sol)

    push!(Omegas, omega)

    Jˢⁱᵈ = [DenseOperator(basis, j) for j in subs.(Jˢʸᵐᵇ[1:op_num], sol)]
    
    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, γᵢ) # create gamma coupling group   

        γ_group["omega"] = convert(Float64, omega)

        if op_num >= 1 
            relaxation = subs(r^2, sol)
            γ_group["gamma_relaxation"] = convert(Float64, relaxation)
        end
        
        γ_group["H"] = convert.(ComplexF64, Hˢⁱᵈ)

        for o in 1:op_num
            γ_group["J"*string(o)] = convert.(ComplexF64, subs.(Jˢʸᵐᵇ[o], sol))
        end

    end

    FminStates = []
    FmedianStates = []
    FmeanStates = []

    for state in test_states # loop over initial states
        
        print(state*" ")

        start_time = time()

        tₛ, ρₛ = read_timeevolution(evol_data_file_name, state, γᵢ)
        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        bᵗˢᵗ = LiPoSID.bloch(ρₛ)
        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]
        tᵗˢᵗ = convert(Vector{Float64}, tₛ)

        tˢⁱᵈ, ρˢⁱᵈ  = timeevolution.master(tᵗˢᵗ, ρᵗˢᵗ[1], DenseOperator(basis, Hˢⁱᵈ), Jˢⁱᵈ)
        bˢⁱᵈ = LiPoSID.bloch([ρₜ.data for ρₜ in ρˢⁱᵈ])

        Fᴸᴹᴱₑₓ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈ)]   

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, γᵢ) # open gamma coupling group
            init_state_group = create_group(γ_group, state) # create initial state group
            init_state_group["Fidelity"] = convert.(Float64, Fᴸᴹᴱₑₓ)
            init_state_group["bloch_exact"] = convert.(Float64, bᵗˢᵗ)
            init_state_group["bloch_sid"] = convert.(Float64, bˢⁱᵈ)
            init_state_group["tr_dist_grnd"] = TrDist(ρₛ[1], ρᵍ₀)
        end
        
        FminState = minimum(Fᴸᴹᴱₑₓ)
        FmedianState = mean(Fᴸᴹᴱₑₓ)
        FmeanState = mean(Fᴸᴹᴱₑₓ)
        
        push!(FminStates, FminState)
        push!(FmedianStates, FmedianState)
        push!(FmeanStates, FmeanState)
    
    end

    # Calculate the mean
    F_mean_value = mean(FmeanStates)

    # Calculate the median
    F_median_value = median(FmedianStates)

    # Calculate the min
    F_min_value = minimum(FminStates)

    push!(FminGammas, F_min_value)
    push!(FmedianGammas, F_median_value)
    push!(FmeanGammas, F_mean_value)

    println("Median fidelity for "*γᵢ*": ", F_median_value)

end

h5open(tests_dir*tests_data_file_name,"cw") do fid
    fid["F_min"] = convert.(Float64,FminGammas)
    fid["F_median"] = convert.(Float64,FmedianGammas)
    fid["F_mean"] = convert.(Float64,FmeanGammas)
    fid["Omegas"] = convert.(Float64,Omegas)
end

println(tests_data_file_name)

γ =  0.079477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.079477: 0.9996374593460027
γ =  0.25133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.25133: 0.9999366287455128
γ =  0.79477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 0.79477: 0.9994696355117969
γ =  2.5133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 2.5133: 0.9975362003916117
γ =  7.9477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 7.9477: 0.9917292226023138
γ =  25.133
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 25.133: 0.9647406039459694
γ =  79.477
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 79.477: 0.92094032075368
γ =  251.33
D1 D2 D3 D4 D5 D6 D7 D8 D9 D10 Median fidelity for 251.33: 0.8825657253257567
POP_NEW_LME_AMZ1_op1_random_trn4_tst10_2024-Mar-14_at_20-28.h5
